<a href="https://colab.research.google.com/github/lordoz234/ML_Homework/blob/main/%D0%94%D0%BE%D0%BF%D0%BE%D0%BB%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook  
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [32]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"lordozvlad","key":"fe816606c5617e40ec838a122df91cae"}'}

In [33]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [34]:
! cp kaggle.json ~/.kaggle/

In [35]:
! chmod 600 ~/.kaggle/kaggle.json

In [36]:
!kaggle competitions download -c advanced-dls-spring-2021

train.csv: Skipping, found more recently modified local copy (use --force to force download)
submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [37]:
data = pd.read_csv("train.csv")

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientPeriod              5282 non-null   int64  
 1   MonthlySpending           5282 non-null   float64
 2   TotalSpent                5282 non-null   object 
 3   Sex                       5282 non-null   object 
 4   IsSeniorCitizen           5282 non-null   int64  
 5   HasPartner                5282 non-null   object 
 6   HasChild                  5282 non-null   object 
 7   HasPhoneService           5282 non-null   object 
 8   HasMultiplePhoneNumbers   5282 non-null   object 
 9   HasInternetService        5282 non-null   object 
 10  HasOnlineSecurityService  5282 non-null   object 
 11  HasOnlineBackup           5282 non-null   object 
 12  HasDeviceProtection       5282 non-null   object 
 13  HasTechSupportAccess      5282 non-null   object 
 14  HasOnlin

In [39]:
data = pd.read_csv("train.csv")

In [40]:
!pip install eli5

In [41]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestClassifier

In [42]:
data['TotalSpent'] = data['TotalSpent'].where(data['ClientPeriod'] > 0, data['MonthlySpending'])
data['ClientPeriod'] = data['ClientPeriod'].where(data['ClientPeriod'] > 0, 1)
data['TotalSpent'] = pd.to_numeric(data['TotalSpent'])

In [43]:
cat_cols = [x for x in data.columns if data[x].dtype == 'object']
num_cols = [x for x in data.columns if data[x].dtype in ['int64', 'float64'] and x != 'Churn']
target_col = "Churn"
print("Категориальные признаки: ", ", ".join(cat_cols))
print("Числовые признаки: ", ", ".join(num_cols))
print("Целевой признак: ", target_col)

Категориальные признаки:  Sex, HasPartner, HasChild, HasPhoneService, HasMultiplePhoneNumbers, HasInternetService, HasOnlineSecurityService, HasOnlineBackup, HasDeviceProtection, HasTechSupportAccess, HasOnlineTV, HasMovieSubscription, HasContractPhone, IsBillingPaperless, PaymentMethod
Числовые признаки:  ClientPeriod, MonthlySpending, TotalSpent, IsSeniorCitizen
Целевой признак:  Churn


In [137]:
X = data[num_cols + cat_cols].copy()
y = data['Churn'].copy()

In [45]:
from sklearn.preprocessing import MaxAbsScaler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

num_transform = Pipeline(steps=[
                    ('scale', MaxAbsScaler())                             
])
cat_transfrom = Pipeline(steps=[
                    ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
                    ('scale', MaxAbsScaler())
])

In [46]:
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(transformers = [
                                    ('cat', cat_transfrom, cat_cols),
                                    ('num', num_transform, num_cols)
])

In [47]:
from sklearn.preprocessing import MaxAbsScaler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

num_transform = Pipeline(steps=[
                    ('scale', MaxAbsScaler())                             
])
cat_transfrom = Pipeline(steps=[
                    ('onehot', OneHotEncoder(handle_unknown = 'ignore')),
                    ('scale', MaxAbsScaler())
])

In [48]:
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(transformers = [
                                    ('cat', cat_transfrom, cat_cols),
                                    ('num', num_transform, num_cols)
])

In [138]:
X = preprocessing.fit_transform(X)

In [139]:
X = pd.DataFrame(data = X)

In [140]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Feature Importance

In [70]:
modelRF = RandomForestClassifier(random_state = 42).fit(X_train, y_train)
perm = PermutationImportance(modelRF, random_state = 42).fit(X_test, y_test)

In [85]:
eli5.show_weights(perm)

Weight,Feature
0.0335 ± 0.0091,x41
0.0180 ± 0.0049,x43
0.0108 ± 0.0031,x32
0.0091 ± 0.0088,x12
0.0081 ± 0.0079,x23
0.0079 ± 0.0136,x42
0.0068 ± 0.0082,x10
0.0068 ± 0.0048,x17
0.0053 ± 0.0019,x35
0.0051 ± 0.0076,x44


In [83]:
pi_features = eli5.explain_weights_df(perm, feature_names = X_train.columns.tolist())
pi_features = pi_features.loc[pi_features['weight'] >= 0.0001]['feature'].tolist()

In [141]:
pi_features[:5]

[41, 43, 32, 12, 23]

In [86]:
X_train = X_train.iloc[:, pi_features]
X_test  = X_test.iloc[:, pi_features]

In [88]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold, ShuffleSplit, RepeatedKFold, cross_val_predict

# Random Forest

In [89]:
from sklearn.ensemble import RandomForestClassifier

In [90]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=344ae3ffb142606bb44ce781c8cf9bc06e0b18c4ebe460785aecbf3e398bc112
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [91]:
from bayes_opt import BayesianOptimization

In [92]:
from sklearn.metrics import roc_auc_score

In [96]:
def rf (n_estimators, max_features):
    params = {
        'max_features':max_features,
        'n_estimators':int(n_estimators)
    }
    clf = RandomForestClassifier(max_features=params['max_features'], n_estimators=params['n_estimators'])
    clf.fit(X_train, y_train)
    return roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

In [97]:
rf_bo = BayesianOptimization(rf, {
                            'max_features':(0.1, 1),
                            'n_estimators':(100, 1000)
                            })

In [98]:
results = rf_bo.maximize(n_iter = 30, init_points = 5, acq = 'ei')

|   iter    |  target   | max_fe... | n_esti... |
-------------------------------------------------
|  1        |  0.8169   |  0.4219   |  180.3    |
|  2        |  0.8132   |  0.7171   |  512.9    |
|  3        |  0.8186   |  0.1159   |  170.9    |
|  4        |  0.8158   |  0.437    |  775.5    |
|  5        |  0.8196   |  0.322    |  597.8    |
|  6        |  0.8095   |  0.9592   |  597.8    |
|  7        |  0.8115   |  0.5852   |  124.9    |
|  8        |  0.819    |  0.1934   |  620.9    |
|  9        |  0.8218   |  0.1756   |  239.3    |
|  10       |  0.814    |  0.6687   |  554.0    |
|  11       |  0.8206   |  0.157    |  134.1    |
|  12       |  0.8148   |  0.4853   |  436.3    |
|  13       |  0.8153   |  0.5785   |  486.7    |
|  14       |  0.8177   |  0.3592   |  588.8    |
|  15       |  0.8111   |  0.8596   |  514.3    |
|  16       |  0.8218   |  0.2237   |  848.6    |
|  17       |  0.8116   |  0.8604   |  877.1    |
|  18       |  0.8136   |  0.6728   |  640.1    |


In [142]:
params = rf_bo.max['params']
params['n_estimators']= int(params['n_estimators'])

In [143]:
params

{'max_features': 0.175588762229762, 'n_estimators': 239}

In [144]:
clf = RandomForestClassifier(**params)

In [145]:
clf.fit(X, y)

RandomForestClassifier(max_features=0.175588762229762, n_estimators=239)

In [146]:
X_test_origin = pd.read_csv('test.csv')
X_test_origin['TotalSpent'] = X_test_origin['TotalSpent'].where(X_test_origin['ClientPeriod'] > 0, X_test_origin['MonthlySpending'])
X_test_origin['ClientPeriod'] = X_test_origin['ClientPeriod'].where(X_test_origin['ClientPeriod'] > 0, 1)
X_test_origin['TotalSpent'] = pd.to_numeric(X_test_origin['TotalSpent'])
X_test_origin = preprocessing.transform(X_test_origin)
X_test_origin = pd.DataFrame(data = X_test_origin)
submission = pd.read_csv('submission.csv')
submission['Churn'] = clf.predict_proba(X_test_origin)[:, 1]
submission.to_csv('my_submission.csv', index=False)

# Gradient Boosting

In [147]:
from sklearn.ensemble import GradientBoostingClassifier

In [148]:
def gboost(min_samples_split, max_features, learning_rate, max_depth, n_estimators):
    params = {
        'min_samples_split': int(min_samples_split),
        'max_features': max_features,
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'n_estimators': int(n_estimators)
    }
    clf = GradientBoostingClassifier(**params)
    return np.mean(cross_val_score(clf, X, y, scoring='roc_auc'))

In [149]:
gboost = BayesianOptimization(gboost, {
                            'min_samples_split': (2, 5),
                            'max_features':(0.1, 1),
                            'learning_rate': (0.001, 0.1),
                            'max_depth': (2, 5),
                            'n_estimators': (200, 300)
                            })

In [150]:
results = gboost.maximize(n_iter = 50, init_points = 10, acq = 'ei')

|   iter    |  target   | learni... | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.8351   |  0.09575  |  3.709    |  0.3646   |  4.242    |  284.5    |
|  2        |  0.8465   |  0.06073  |  2.265    |  0.2987   |  4.72     |  253.8    |
|  3        |  0.8346   |  0.07659  |  4.717    |  0.4352   |  2.428    |  258.5    |
|  4        |  0.8364   |  0.08668  |  3.734    |  0.8471   |  3.815    |  233.4    |
|  5        |  0.8364   |  0.05859  |  4.08     |  0.692    |  3.439    |  249.8    |
|  6        |  0.8406   |  0.04188  |  4.004    |  0.6003   |  2.633    |  211.9    |
|  7        |  0.8404   |  0.0626   |  3.21     |  0.5313   |  4.439    |  291.7    |
|  8        |  0.8472   |  0.01827  |  2.624    |  0.2887   |  4.123    |  276.6    |
|  9        |  0.8334   |  0.07854  |  4.033    |  0.3728   |  3.161    |  277.7    |
|  10       |  0.8328   |  0.08352  |  4.781    |  0.6

In [151]:
params = gboost.max['params']
params['n_estimators']= int(params['n_estimators'])
params['max_depth']= int(params['max_depth'])
params['min_samples_split']= int(params['min_samples_split'])

In [152]:
params

{'learning_rate': 0.02938215271491487,
 'max_depth': 2,
 'max_features': 0.67961686099773,
 'min_samples_split': 4,
 'n_estimators': 252}

In [153]:
clf = GradientBoostingClassifier(**params)

In [154]:
clf.fit(X, y)

GradientBoostingClassifier(learning_rate=0.02938215271491487, max_depth=2,
                           max_features=0.67961686099773, min_samples_split=4,
                           n_estimators=252)

In [155]:
X_test_origin = pd.read_csv('test.csv')
X_test_origin['TotalSpent'] = X_test_origin['TotalSpent'].where(X_test_origin['ClientPeriod'] > 0, X_test_origin['MonthlySpending'])
X_test_origin['ClientPeriod'] = X_test_origin['ClientPeriod'].where(X_test_origin['ClientPeriod'] > 0, 1)
X_test_origin['TotalSpent'] = pd.to_numeric(X_test_origin['TotalSpent'])
X_test_origin = preprocessing.transform(X_test_origin)
X_test_origin = pd.DataFrame(data = X_test_origin)
submission = pd.read_csv('submission.csv')
submission['Churn'] = clf.predict_proba(X_test_origin)[:, 1]
submission.to_csv('my_submission.csv', index=False)